# <center> **Webscraping**

## **Présentation**


- Webscraping du site [www.allocine.fr](https://www.allocine.fr/films/) avec Beautiful Soup et Selenium.

Nous vérifions tout d'abord le fichier "robots.txt" pour voir si nous sommes autorisés à scraper le film.

![robots.txt](images/robots.txt.png)

Il n'y a pas de limitation pour notre tâche puisque nous travaillons sous l'url : <code>https://www.allocine.fr/film/</code><br>

Ensuite nous allons sur le site allocine, catégorie **films** et nous allons scraper les informations à partir des menus déroulants sur la gauche (les catégories de films, les pays et ensuite nous scraperons les films par année).

![filtres](images/filtresSMALL.png)
### Sources :
**Beautiful Soup** :
[beautiful-soup-4](https://beautiful-soup-4.readthedocs.io/en/latest/)<br>
[beautiful-soup-4.readthedocs.io](https://beautiful-soup-4.readthedocs.io/en/latest/#searching-the-tree)<br>

**Selenium** :<br>
[selenium-python.readthedocs.io](https://selenium-python.readthedocs.io/locating-elements.html)<br>
[selenium.dev/documentation](https://www.selenium.dev/documentation/webdriver/elements/information/)<br>
[selenium.dev/documentation/finders/](https://www.selenium.dev/documentation/webdriver/elements/finders/)<br>
[geeksforgeeks.org/get_property-selenium/](https://www.geeksforgeeks.org/get_property-element-method-selenium-python/)<br>

Les liens sont sûrement générés aléatoirement dynamiquement, on peut utiliser XPath avec selenium<br>
ou bien avec lxml ??<br>

Sur ce lien https://medium.com/swlh/web-scraping-using-selenium-and-beautifulsoup-adfc8810240a Selenium est utilisé pour faire le scraping des urls puis ensuite beautiful soup est utilisé pour faire le scraping des pages de chaque urls.

## **Imports**

In [1]:
%reset

In [14]:
import os
# import re
import io
import math
import copy
import httpx
import requests
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
from IPython.display import display
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

%config IPCompleter.greedy = True

url_site  = 'https://www.allocine.fr/'
url_films = 'https://www.allocine.fr/films/'

## **Scraping**

### **On scrape la liste des genres de film**

In [18]:
# Scrap all categories
r = requests.get(url_films, auth=('user', 'pass'))
if r.status_code != 200:
    print("url_site error")
    
soup = BeautifulSoup(r.content, 'html.parser')
print(type(soup))

categories = []
elt_categories = soup.find('div', class_='filter-entity-section')
for elt in elt_categories.find_all('li'):
    #print(elt.prettify())
    categories.append(elt.a.text)

print("Nb categories :", len(categories))
df_categories = pd.Series(categories)

dict_n_cat = {k:v for k, v in enumerate(categories)}
dict_cat_n = {v:k for k, v in dict_n_cat.items()}

<class 'bs4.BeautifulSoup'>
Nb categories : 37


### **On scrape la liste des pays d'origine des films**

In [21]:
# Scrap all countries
elt_countries = elt_categories.find_next_sibling().find_next_sibling()
elts_items = elt_countries.find_all('li', class_ = 'filter-entity-item')

countries = []
for elt_item in elts_items:
    countries.append(elt_item.find('span').text.strip())

print(countries)

print("Nb pays :", len(countries))
df_countries = pd.Series(countries)

dict_n_countries = {k:v for k, v in enumerate(countries)}
dict_countries_n = {v:k for v, k in dict_n_countries.items()}

['France', 'U.S.A.', 'Afrique du Sud', 'Albanie', 'Algérie', 'Allemagne', "Allemagne de l'Est", "Allemagne de l'Ouest", 'Arabie Saoudite', 'Argentine', 'Arménie', 'Australie', 'Autriche', 'Belgique', 'Bengladesh', 'Bolivie', 'Bosnie-Herzégovine', 'Brésil', 'Bulgarie', 'Burkina Faso', 'Cambodge', 'Cameroun', 'Canada', 'Chili', 'Chine', 'Chypre', 'Colombie', 'Corée', 'Corée du Sud', 'Croatie', 'Cuba', "Côte-d'Ivoire", 'Danemark', 'Egypte', 'Emirats Arabes Unis', 'Espagne', 'Estonie', 'Finlande', 'Grande-Bretagne', 'Grèce', 'Géorgie', 'Hong-Kong', 'Hongrie', 'Inde', 'Indonésie', 'Irak', 'Iran', 'Irlande', 'Islande', 'Israël', 'Italie', 'Japon', 'Jordanie', 'kazakhstan', 'Kenya', 'Kosovo', 'Lettonie', 'Liban', 'Lituanie', 'Luxembourg', 'Macédoine', 'Malaisie', 'Maroc', 'Mexique', 'Monténégro', 'Nigéria', 'Norvège', 'Nouvelle-Zélande', 'Pakistan', 'Palestine', 'Pays-Bas', 'Philippines', 'Pologne', 'Portugal', 'Pérou', 'Qatar', 'Roumanie', 'Russie', 'République dominicaine', 'République tchè

### **On scrape les liens redirigeants vers les pages de films par année**

**Nous utilisons Selenium**<br>
Lors de l'utilisation de Beautiful Soup, certains éléments sont **décorés**, certains liens sont **invisibles**, on ne peut pas directement les scraper.<br>
Le contournement trouvé est d'utiliser Selenium qui permet entre autre :
- d'utiliser les XPath,
- de récupérer tous les élements et non-décorés.

On se donne une liste d'années, par exemple [1980, ... 2000]

[1980 - 1989] puis [1990 - 1999] ... jusqu'à [2000 - 2009].<br>
(cela fait plus de 40000 films et XXX reviews).

**Liens visualisés dans l'inspecteur html de Chrome**

![links_decades_inspector](images/link_decades_inspector.png)


**Liens visualisés avec Beautiful Soup**

![link](images/link_decades_bs4.png)

In [ ]:
# To show the limit of Beautiful Soup
r = requests.get(url_films, auth=('user', 'pass'))
if r.status_code != 200:
    print("url_site error")
    
soup = BeautifulSoup(r.content, 'html.parser')
# print(soup.prettify())

elt_decades = elt_categories.find_next_sibling()
print(elt_decades.prettify())

In [82]:
# Scrap the links of the years

# Input: list of years to scrap
lst_years_to_scrap = list(range(1980, 2009))
lst_years_to_scrap = [1980]

lst_decades_to_scrap = list(set([10 * (year // 10) for year in lst_years_to_scrap]))
lst_years_to_scrap = [str(year) for year in lst_years_to_scrap]
lst_decades_to_scrap = [str(decade) for decade in lst_decades_to_scrap]

driver = webdriver.Chrome(options = options)
driver.get(url_films)
elts_decades = driver.find_elements(By.XPATH, '/html/body/div[2]/main/section[4]/div[1]/div/div[3]/div[2]/ul/li')

dict_year_link = {}
for elt_decade in tqdm(elts_decades):
    elt_a = elt_decade.find_element(By.TAG_NAME, 'a')
    if not(elt_a.get_attribute('title')[:4] in lst_decades_to_scrap):
        continue

    driver2 = webdriver.Chrome(options = options)
    url_decade = elt_a.get_attribute('href').strip()

    driver2.get(url_decade)
    elts_years = driver2.find_elements(By.XPATH, '/html/body/div[2]/main/section[4]/div[1]/div/div[3]/div[3]/ul/li')

    for elt_year in elts_years:
        year = elt_year.find_element(By.TAG_NAME, 'a').get_attribute('title').strip()
        if year in lst_years_to_scrap:
            link = elt_year.find_element(By.TAG_NAME, 'a').get_attribute('href').strip()
            dict_year_link[year] = link
    driver2.close()

for year, url_year in dict_year_link.items():
    print("year", year, '  ----  link', url_year)

driver.close()

100%|██████████| 15/15 [00:02<00:00,  6.14it/s]

year 1980   ----  link https://www.allocine.fr/films/decennie-1980/annee-1980/


### **On scrape les films à partir des liens vers les années**


Pour scraper les directeurs et acteurs nous allons sur la page **casting** du film puis nous scrapons les acteurs principaux représentés dans la mosaïque, ensuite nous scrapons la liste des acteurs secondaires.

![all_actors](images/scraping_all_actors.png)

On observe comment récupérere l'url de la page des films similaires en fonction de la page du film :<br>
https://www.allocine.fr/film/fichefilm_gen_cfilm=180.html<br>
https://www.allocine.fr/film/fichefilm-180/similaire/<br>

Cela suit toujours le même modèle, nous allons pouvoir automatiser cela sans scraper les urls.

In [ ]:
def number_pages_per_year(soup_year):
    ''' Return the number of pages for one year'''
    pagination = soup_year.find('div', class_='pagination-item-holder')
    nb_pages = int(pagination.find_all('span')[-1].text)
    return int(nb_pages)

def delete_thumbnails():
    '''Delete all files in thumbnail directory'''
    try:
        folder_name = os.getcwd() + '\\thumbnails\\'
        files = os.listdir(folder_name)
        for file in files:
            file_path = os.path.join(folder_name, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("All files deleted successfully.")
    except OSError:
        print("Error occurred while deleting files.")

def get_title(soup_movie):
    ''' Return the title of the movie '''
    return soup_movie.find('div', class_ = "titlebar-title titlebar-title-xl").text

def get_date_duration_categories(soup_movie):
    ''' Return date, duration and categories (as string) of the movie '''
    elt = soup_movie.find('div', class_="meta-body-item meta-body-info")
    text = elt.get_text(strip=True)
    s1, s2, s3 = text.split('|')
    date = s1[:-8].strip()
    duration = s2.strip()
    categories = s3.strip()
    return date, duration, categories

def get_directors(soup_casting):
    ''' Return list of directors '''
    elt_director_section = soup_casting.find('section', class_='section casting-director')
    elt_temp = elt_director_section.find_next()
    elts_directors = elt_temp.find_next_sibling().find_all('div', class_ = 'card person-card person-card-col')
    lst_directors = [elt_director.find('a').text for elt_director in elts_directors]
    return lst_directors

def get_actors(soup_casting):
    ''' Return list of actors (maximum 30) '''
    elt_actor_section = soup_casting.find('section', class_ = 'section casting-actor')
    if not(elt_actor_section):
        return []
    elt_temp = elt_actor_section.find_next()
    # scrap main actors (maximum eight actors in the mosaic, see image above)
    elts_actors = elt_temp.find_next_sibling().find_all('div', class_ = 'card person-card person-card-col')
    lst_actors = [elt_actor.find('figure').find('span')['title'] for elt_actor in elts_actors]
    elts_actors = elt_actor_section.find_all('div', class_ = 'md-table-row')
    # scrap list of actors below the mosaic (we scrap maximum of (8 + 22) 30 actors in total)
    lst_actors.extend([elt_actor.find('a').text for elt_actor in elts_actors[:22] if elt_actor.find('a')])
    return lst_actors

def get_composers(soup_casting):
    ''' Scrap the name(s) of the music composer(s) '''
    elts_sections = soup_casting.find_all("div", class_ = "section casting-list-gql")
    for elt_section in elts_sections:
        elt_title = elt_section.find('div', class_ = 'titlebar section-title').find('h2')
        if 'Soundtrack' in elt_title.text:
            lst_composers = []
            elts_composers = elt_section.find_all('div', class_ = 'md-table-row')
            for elt_composer in elts_composers:
                elts_span = elt_composer.find_all('span')
                if len(elts_span) > 1 and 'Compositeur' in elts_span[1].text.strip():
                    lst_composers.append(elts_span[0].text.strip())
            return lst_composers

def get_summary(soup_movie):
    elt = soup_movie.find('section', class_ = "section ovw ovw-synopsis")
    return elt.find('p', class_ = 'bo-p').text.strip()

def get_thumbnail(soup_movie):
    elt = soup_movie.find('figure', class_ = 'thumbnail')
    return elt.span.img['src']

def save_thumbnail(title, url_thumbnail):
    '''Save the thumbnail as image file in directory "thumbnails"'''
    try:
        folder_name = os.getcwd() + '\\thumbnails\\'
        title2 = title.replace('-', '')
        image_name = f"thumbnail-{title2}.jpg"
        file = open(folder_name + image_name, "wb")
        image = httpx.get(url_thumbnail)
        file.write(image.content)
        # Display thumbnail in Jupyter / console
        img = Image.open(io.BytesIO(image.content))
        plt.imshow(img)
        plt.axis('off')
        plt.show()
        # To change resolution: https://www.geeksforgeeks.org/change-image-resolution-using-pillow-in-python/
    except IOError:
        print("Cannot read the file")
    finally:
        file.close()

def get_similar_movies(url_similar_movies):
    print('url_similar_movies:', url_similar_movies)
    pass

def scrap_movie(elt_movie, driver):
    ''' scrap all movie informations '''
    
    # get the movie soup
    url_movie = url_site + elt_movie.h2.a.get('href')[1:]
    r = requests.get(url_movie, auth=('user', 'pass'))
    soup_movie = BeautifulSoup(r.content, 'html.parser')

    # ------------- #
    #     Title     #
    # ------------- #
    title = get_title(soup_movie)
    print("Title:" , title)

    # --------------------------------- #
    #   Date, duration and categories   #
    # --------------------------------- #
    date, duration, categories = get_date_duration_categories(soup_movie)
    print("Date:", date)
    print("Duration:", duration)
    print("Categories:", categories)
    
    if 'Documentaire' in categories:
        print('Documentaire and more categories', categories)
        # return (We do not scrap the documentaries ???)

    # ------------------------------------------------- #
    #   Directors list / Actors list / Composers list   #
    # ------------------------------------------------- #
    lst_directors, lst_actors, lst_composers = [], [], []
    elts_end_section = soup_movie.find_all('a', class_ = 'end-section-link')
    
    for elt_end_section in elts_end_section:
        if 'Casting' in elt_end_section['title']:
            # If there is a link to the whole casting
            link_casting = elt_end_section['href']
            r = requests.get(url_site + link_casting, auth=('user', 'pass'))
            soup_casting = BeautifulSoup(r.content, 'html.parser')

            # Get directors' list
            lst_directors = get_directors(soup_casting)
            # Get actors' list
            lst_actors = get_actors(soup_casting)
            # Composers' list
            lst_composers = get_composers(soup_casting)

        else: # No link to the casting 
              # for example animation movies does not have a casting section
              # some movies neither: 

            # print(elt_end_section.prettify())
            elts = soup_movie.find_all('div', class_ = "meta-body-item meta-body-direction meta-body-oneline")

            # if len(elts):
            # Get directors' list
            lst_directors = [elts[0].text.strip()[2:].strip()]
            # For this kind of movies we only scrap one director (to see ...) and no actor, no composer

    print('Directors:', lst_directors)
    print('Actors:', lst_actors)
    print('Composers:', lst_composers)

    # ------------ #
    #   Summary    #
    # ------------ #
    summary = get_summary(soup_movie)[:180]
    print("Summary:", summary)

    # ------------ #
    #   Thumbnail  #
    # ------------ #
    url_thumbnail = get_thumbnail(soup_movie)
    print("Thumbnail:", url_thumbnail)
    # save_thumbnail(title, url_thumbnail)

    # ------------ #
    #    Ratings   #
    # ------------ #
    star_rating, nb_notes, nb_critics = get_ratings(soup_movie, driver, False)
    print("star rating:", star_rating, nb_notes, nb_critics)

    # ------------------- #
    #     url_reviews     #
    # ------------------- #
    url_reviews = url_movie.replace('_gen_cfilm=', '-')[:-5] + '/critiques/spectateurs/'
    print(url_reviews)

    # ------------------- #
    #    Similar Movies   #
    # ------------------- #
    url_similar_movies = url_movie.replace('_gen_cfilm=', '-')[:-5] + '/similaire/' 
    # soup_similar_movies
    get_similar_movies(url_similar_movies)
    return

def get_ratings(soup_movie, driver, use_Selenium):
    ''' Scrap the ratings of the movie.

        Return:
         - stareval:   star rating (0.5 to 5),
         - nb_notes:   number of votes,
         - nb_critics: number of critics (written reviews),

        Args:
         - soup_movie:   object BeautifulSoup of the movie,
         - driver:       Selenium driver,
         - use_Selenium: boolean to choose the method to scrape,
                         True:  Selenium's method,      (SLOWER)
                         False: Beautifulsoup's method. (FASTER)
    '''

    star_rating, nb_notes, nb_critics = None, None, None

    if use_Selenium:
        elts_ratings = driver.find_elements(By.CLASS_NAME, 'rating-item')

        for elt_rating in elts_ratings:
            elt = None
            try:
                elt = elt_rating.find_element(By.TAG_NAME, 'a')
                if 'Spectateurs' in elt.text.strip():
#                    url_reviews = elt.get_attribute('href')
                    elt_stareval_note = elt_rating.find_element(By.CLASS_NAME, 'stareval-note')
                    star_rating = elt_stareval_note.text.strip()
                    elt_stareval_review = elt_rating.find_element(By.CLASS_NAME, 'stareval-review')
                    stareval_review = elt_stareval_review.text.strip()
                    assert stareval_review.count(',') == 1
                    nb_notes, nb_critics = stareval_review.split(',')
                    nb_notes = nb_notes.split()[0]
                    nb_critics = nb_critics.split()[0]
            except:
                print('no tag "a" in elt_rating')

    else: # user beautiful soup
        elts_ratings = soup_movie.find_all('div', class_ = 'rating-item-content')
        elts_ratings[0].find
        for elt_rating in elts_ratings:
            if 'Spectateurs' in elt_rating.find("span").text.strip():
                elt_stareval_note = elt_rating.find("span", class_ = "stareval-note")
                star_rating = elt_stareval_note.text.strip()
                elt_stareval_review = elt_rating.find("span", class_ = "stareval-review")
                stareval_review = elt_stareval_review.text.strip()
                assert stareval_review.count(',') == 1
                nb_notes, nb_critics = stareval_review.split(',')
                nb_notes = nb_notes.split()[0]
                nb_critics = nb_critics.split()[0]

    return star_rating, nb_notes, nb_critics


# ---------------------------------- #
#                                    #
#             Main loop              #
#                                    #
# ---------------------------------- #
        

# loop on all years to scrap (through links previously scrapped)
# then loop on all pages of the year
# then loop on all movies on one page
# 

# delete_thumbnails()

# Create Selenium driver
# driver.close()
# driver = webdriver.Chrome(options = options)
driver = None

for year, url_year in dict_year_link.items():
    print("Scraping year:", year)
    
    r = requests.get(url_year, auth=('user', 'pass'))
    if r.status_code != 200:
        print("url_site error")

    soup_year = BeautifulSoup(r.content, 'html.parser')
    nb_pages = number_pages_per_year(soup_year)
    # print("Nb pages :", nb_pages)

    for i in range(nb_pages): # Need to reduce as some movies are totaly unknown with very few informations about
        url_year_page = url_year + f'?page={i+1}'
        r = requests.get(url_year_page, auth=('user', 'pass'))
        if r.status_code != 200:
            print("url_site error")

        soup_movies = BeautifulSoup(r.content, 'html.parser')
        elt_movies = soup_movies.find_all('li', class_='mdl')
        for elt_movie in elt_movies:
            print('---------------------------------------------------------------- ')
            scrap_movie(elt_movie, driver)

Scraping year: 1980
---------------------------------------------------------------- 
Title: Elephant Man
Date: 8 avril 1981
Duration: 2h 05min
Categories: Biopic,Drame
Directors: ['David Lynch']
Actors: ['photo de Anthony Hopkins', 'photo de John Hurt', 'photo de Anne Bancroft', 'photo de John Gielgud', 'photo de Wendy Hiller', 'photo de Freddie Jones', 'photo de Hannah Gordon', 'photo de Michael Elphick', 'Lesley Dunlop', 'Kenny Baker', 'John Standing', 'Dexter Fletcher', 'Phoebe Nicholls', 'Patsy Smart', 'Frederick Treves', 'Roy Evans', 'Tony London']
Composers: ['John Morris (II)']
Summary: Londres, 1884. Le chirurgien Frederick Treves découvre un homme complètement défiguré et difforme, devenu une attraction de foire. John Merrick, " le monstre ", doit son nom de Ele
Thumbnail: https://fr.web.img4.acsta.net/c_310_420/pictures/20/02/21/16/48/4302324.jpg
star rating: 4,4 29339 689
https://www.allocine.fr/film/fichefilm-180/critiques/spectateurs/
url_similar_movies: https://www.alloc

IndexError: list index out of range